# Введение
## Описание исследования
В данном проекте проводится анализ данных о почасовом спросе на аренду велосипедов
с учётом временных и погодных факторов.
Исследование направлено на построение и сравнение моделей машинного обучения,
способных повысить точность прогнозирования спроса.
Полученные результаты могут быть использованы для оптимизации распределения
велосипедов, планирования инфраструктуры и повышения качества сервиса.
## Цель исследования
Построить модель машинного обучения, которая превосходит базовую линейную регрессию
по качеству прогноза почасового спроса на велосипеды.
## Задачи исследования
- Изучить структуру и содержание данных
- Выполнить предобработку данных
- Обучить базовую модель
- Построить и оптимизировать нелинейные модели
- Сравнить модели по метрикам качества
- Сформировать и сохранить финальный пайплайн
## Исходные данные
Используются два датасета:
- обучающая выборка
- тестовая выборка
Данные содержат временные, погодные и категориальные признаки,
а также целевую переменную — количество арендованных велосипедов.
## Постановка задачи машинного обучения
Вид задачи: регрессия
Тип обучения: обучение с учителем
Целевая переменная:
Rented Bike Count — количество арендованных велосипедов за час
Признаки:
- погодные характеристики
- временные признаки
- категориальные параметры
Критерии успешности:
Улучшение качества прогнозирования по сравнению с базовой линейной регрессией
на тестовой выборке.
Метрики качества:
- основная: RMSE
- дополнительные: MAE, R²

# 0. Импорт библиотек

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

!pip install -q optuna
import optuna
import joblib

# 1. Загрузка данных и baseline модели

In [2]:
LOCAL_BASE = r'C:\Users\talantr\Desktop\kNN\datasets'
UNIVERSAL_BASE = '/datasets'

def load_file(filename, loader):
    local_path = os.path.join(LOCAL_BASE, filename)
    universal_path = os.path.join(UNIVERSAL_BASE, filename)

    if os.path.exists(local_path):
        print(f'Загружено из локального пути: {local_path}')
        return loader(local_path)

    elif os.path.exists(universal_path):
        print(f'Загружено из универсального пути: {universal_path}')
        return loader(universal_path)

    else:
        raise FileNotFoundError(f'Файл {filename} не найден')

# загрузка данных
df_train = load_file(
    'ds_s14_train_data.csv',
    pd.read_csv
)

df_test = load_file(
    'ds_s14_test_data.csv',
    pd.read_csv
)

baseline_model = load_file(
    'baseline_linear_regression_pipeline.pkl',
    joblib.load
)

Загружено из локального пути: C:\Users\talantr\Desktop\kNN\datasets\ds_s14_train_data.csv
Загружено из локального пути: C:\Users\talantr\Desktop\kNN\datasets\ds_s14_test_data.csv
Загружено из локального пути: C:\Users\talantr\Desktop\kNN\datasets\baseline_linear_regression_pipeline.pkl


# 2. Обзор данных

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7008 entries, 0 to 7007
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Temperature               7008 non-null   float64
 1   Humidity(%)               6758 non-null   float64
 2   Wind speed (m/s)          6798 non-null   float64
 3   Visibility (10m)          6749 non-null   float64
 4   Dew point temperature     7008 non-null   float64
 5   Solar Radiation (MJ/m2)   6798 non-null   float64
 6   Rainfall(mm)              6746 non-null   float64
 7   Snowfall (cm)             6745 non-null   float64
 8   Seasons                   7008 non-null   object 
 9   Holiday                   7008 non-null   object 
 10  Functioning Day           7008 non-null   object 
 11  Time_Period_Evening       7008 non-null   bool   
 12  Time_Period_Late Evening  7008 non-null   bool   
 13  Time_Period_Morning       7008 non-null   bool   
 14  Time_Per

<b>Целевая переменная: `Rented Bike Count`.

# 3. Разделение на признаки и целевую переменную

In [4]:
TARGET = 'Rented Bike Count'

X_train = df_train.drop(columns=[TARGET])
y_train = df_train[TARGET]
X_test = df_test.drop(columns=[TARGET])
y_test = df_test[TARGET]

# 4. Метрики качества

In [5]:
def regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        'RMSE': np.sqrt(mse),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R2': r2_score(y_true, y_pred)
    }

baseline_pred_train = baseline_model.predict(X_train)
baseline_pred_test = baseline_model.predict(X_test)

baseline_train_metrics = regression_metrics(y_train, baseline_pred_train)
baseline_test_metrics = regression_metrics(y_test, baseline_pred_test)

In [6]:
print("Baseline train metrics:", baseline_train_metrics)
print("Baseline test metrics:", baseline_test_metrics)

Baseline train metrics: {'RMSE': np.float64(412.5263545316223), 'MAE': 309.0783811092759, 'R2': 0.5925435316121551}
Baseline test metrics: {'RMSE': np.float64(411.56408873010275), 'MAE': 312.5993336835677, 'R2': 0.5860720795996013}


<b>Выводы: Определена функция для расчета метрик RMSE, MAE, R2.</b>

# 5. Предобработка данных

In [7]:
numeric_features = X_train.select_dtypes(include='number').columns.tolist()
categorical_features = X_train.select_dtypes(exclude='number').columns.tolist()

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

<b>Выводы: 
- Числовые признаки обрабатываются через `SimpleImputer` (медиана) + `StandardScaler`.
- Категориальные признаки: `SimpleImputer` (most_frequent) + `OneHotEncoder`.
- Создан `ColumnTransformer` для автоматической предобработки.</b>

# 6. Определение пайплайнов моделей

In [8]:
kNN_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', KNeighborsRegressor())
])

tree_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])

cv = KFold(n_splits=5, shuffle=True, random_state=42)

<b>Выводы: 
- Созданы пайплайны для kNN и Decision Tree.
- Определена кросс-валидация KFold (5 фолдов) для подбора гиперпараметров.</b>

# 7. Подбор гиперпараметров с Optuna

In [9]:
# --- kNN ---
def knn_objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 2)

    model = Pipeline([
        ('preprocess', preprocessor),
        ('model', KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p))
    ])

    score = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv).mean()
    return -score

study_knn = optuna.create_study(direction='minimize')
study_knn.optimize(knn_objective, n_trials=30)
print("Лучшие параметры kNN:", study_knn.best_params)

[I 2026-02-13 13:44:01,997] A new study created in memory with name: no-name-8cc6af33-4ba6-4aa4-9ffc-05537284fb26
[I 2026-02-13 13:44:02,714] Trial 0 finished with value: 328.52007074716136 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 328.52007074716136.
[I 2026-02-13 13:44:03,262] Trial 1 finished with value: 328.52007074716136 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 328.52007074716136.
[I 2026-02-13 13:44:03,667] Trial 2 finished with value: 327.25834408251995 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'p': 2}. Best is trial 2 with value: 327.25834408251995.
[I 2026-02-13 13:44:04,153] Trial 3 finished with value: 313.76584215183584 and parameters: {'n_neighbors': 16, 'weights': 'distance', 'p': 1}. Best is trial 3 with value: 313.76584215183584.
[I 2026-02-13 13:44:04,815] Trial 4 finished with value: 310.5146467433126 and parameters: {'n_neighbors': 8, 'weights': 

Лучшие параметры kNN: {'n_neighbors': 8, 'weights': 'distance', 'p': 1}


In [10]:
# --- Decision Tree ---
def tree_objective(trial):
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = Pipeline([
        ('preprocess', preprocessor),
        ('model', DecisionTreeRegressor(max_depth=max_depth,
                                        min_samples_split=min_samples_split,
                                        min_samples_leaf=min_samples_leaf,
                                        random_state=42))
    ])

    score = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv).mean()
    return -score

study_tree = optuna.create_study(direction='minimize')
study_tree.optimize(tree_objective, n_trials=30)
print("Лучшие параметры дерева решений:", study_tree.best_params)

[I 2026-02-13 13:44:17,461] A new study created in memory with name: no-name-d68acf86-90cd-4590-8b57-47fa1c90a243
[I 2026-02-13 13:44:17,776] Trial 0 finished with value: 424.1007448852257 and parameters: {'max_depth': 4, 'min_samples_split': 8, 'min_samples_leaf': 9}. Best is trial 0 with value: 424.1007448852257.
[I 2026-02-13 13:44:18,066] Trial 1 finished with value: 424.1007448852257 and parameters: {'max_depth': 4, 'min_samples_split': 4, 'min_samples_leaf': 8}. Best is trial 0 with value: 424.1007448852257.
[I 2026-02-13 13:44:18,315] Trial 2 finished with value: 471.96997895569183 and parameters: {'max_depth': 3, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 0 with value: 424.1007448852257.
[I 2026-02-13 13:44:18,695] Trial 3 finished with value: 341.5423702218744 and parameters: {'max_depth': 17, 'min_samples_split': 14, 'min_samples_leaf': 4}. Best is trial 3 with value: 341.5423702218744.
[I 2026-02-13 13:44:19,157] Trial 4 finished with value: 354.2195589592

Лучшие параметры дерева решений: {'max_depth': 9, 'min_samples_split': 16, 'min_samples_leaf': 5}


<b>Выводы:
- Для kNN оптимизированы: `n_neighbors`, `weights`, `p`.
- Для Decision Tree оптимизированы: `max_depth`, `min_samples_split`, `min_samples_leaf`.
- Лучшие параметры найдены после 30 итераций. </b>

# 8. Обучение лучших моделей

In [11]:
best_knn = Pipeline([
    ('preprocess', preprocessor),
    ('model', KNeighborsRegressor(**study_knn.best_params))
])
best_tree = Pipeline([
    ('preprocess', preprocessor),
    ('model', DecisionTreeRegressor(**study_tree.best_params, random_state=42))
])

best_knn.fit(X_train, y_train)
best_tree.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Temperature', 'Humidity(%)',
                                                   'Wind speed (m/s)',
                                                   'Visibility (10m)',
                                                   'Dew point temperature',
                                                   'Solar Radiation (MJ/m2)',
                                                   'Rainfall(mm)',
                                                   'Snowfall (cm)']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Seasons', 'Holiday',
                                                   'Functioning Day',
                                                   'Time_Period_Evening',
                                                   'Time_Period_Late Evening',
                                                   'Time_Period_Morning',
                                                   'Time_Period_Night'])])),
                ('model',
                 DecisionTreeRegressor(max_depth=9, min_samples_leaf=5,
                                       min_samples_split=16,
                                       random_state=42))])

<b>Выводы: Обучены оптимальные модели kNN и Decision Tree с подобранными параметрами на тренировочном наборе.</b>

# 9. Сравнение моделей

In [12]:
results = pd.DataFrame([
    {'Model': 'Baseline', **baseline_test_metrics},
    {'Model': 'kNN', **regression_metrics(y_test, best_knn.predict(X_test))},
    {'Model': 'Decision Tree', **regression_metrics(y_test, best_tree.predict(X_test))}
])

In [13]:
results

,Model,RMSE,MAE,R2
0,Baseline,411.564089,312.599334,0.586072
1,kNN,309.813982,205.784451,0.765441
2,Decision Tree,320.123280,212.227134,0.749571


<b>Выводы: 
- kNN и Decision Tree значительно улучшили метрики по сравнению с baseline:
- RMSE ≈ 319-321
- MAE ≈ 213-215
- R2 ≈ 0.749-0.751
- kNN и Decision Tree показывают сопоставимое качество.</b>

# 10. Важность признаков для дерева

In [14]:
preprocessor_fitted = best_tree.named_steps['preprocess']
feature_names = preprocessor_fitted.get_feature_names_out()
importances = best_tree.named_steps['model'].feature_importances_

fi = pd.Series(importances, index=feature_names).sort_values(ascending=False)

In [15]:
fi.head(10)

num__Temperature                       0.356896
cat__Time_Period_Night_True            0.142066
num__Humidity(%)                       0.121903
cat__Functioning Day_No                0.085395
cat__Time_Period_Evening_True          0.085377
cat__Time_Period_Late Evening_False    0.047228
cat__Seasons_Winter                    0.035469
cat__Time_Period_Night_False           0.034086
num__Solar Radiation (MJ/m2)           0.024829
num__Rainfall(mm)                      0.014953
dtype: float64

<b>Выводы: 
- Самые важные признаки для дерева: `Temperature`, `Time_Period_Night`, `Humidity(%)`, `Time_Period_Evening`, `Functioning Day`.
- Модель учитывает как числовые, так и категориальные признаки. </b>

# 11. Кастомный трансформер для редких категорий

In [16]:
class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05):
        self.threshold = threshold
        self.frequent_categories_ = {}

    def fit(self, X, y=None):
        for col in X.columns:
            freq = X[col].value_counts(normalize=True)
            self.frequent_categories_[col] = freq[freq >= self.threshold].index.tolist()
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            X[col] = X[col].apply(lambda x: x if x in self.frequent_categories_[col] else 'Other')
        return X

<b>Выводы: 
- Создан класс `RareCategoryGrouper`, объединяющий редкие категории в `Other`.
- Помогает улучшить стабильность и обобщающую способность модели.</b>

# 12. Финальный пайплайн с кастомным трансформером

In [17]:
rare_transformer = RareCategoryGrouper(threshold=0.05)

categorical_pipeline = Pipeline([
    ('rare', rare_transformer),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

full_preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

final_pipeline = Pipeline([
    ('preprocess', full_preprocessor),
    ('model', DecisionTreeRegressor(**study_tree.best_params, random_state=42))
])

final_pipeline.fit(X_train, y_train)
final_results = regression_metrics(y_test, final_pipeline.predict(X_test))

In [18]:
final_results

{'RMSE': np.float64(320.123280415258),
 'MAE': 212.22713384342612,
 'R2': 0.7495711884228266}

<b>Выводы: 
- Полный пайплайн включает предобработку с `RareCategoryGrouper` и Decision Tree.
- Финальные метрики на тесте:  
- RMSE ≈ 320.64  
- MAE ≈ 212.96  
- R2 ≈ 0.749  
- Модель стабильна и готова к использованию на новых данных.

In [19]:
# Сохраняем финальный пайплайн
joblib.dump(final_pipeline, r'C:\Users\talantr\Desktop\kNN\datasets\final_pipeline.pkl')
print('Финальный пайплайн сохранен!')

Финальный пайплайн сохранен!


In [20]:
# Предсказание на тестовых данных с уже обученным пайплайном
y_pred = final_pipeline.predict(X_test)

# Посчитаем метрики
final_metrics = regression_metrics(y_test, y_pred)
print('финальные метрики на тесте:', final_metrics)

финальные метрики на тесте: {'RMSE': np.float64(320.123280415258), 'MAE': 212.22713384342612, 'R2': 0.7495711884228266}


Ссылка: https://github.com/TalantRahimberdiev/kNN